In [2]:
!pip install nltk

In [18]:
!pip install gensim


In [15]:
import numpy as numpy
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abhishek\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhishek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
df = pd.read_csv(r'C:\Users\Abhishek\Downloads\yt_sent.csv', encoding='latin-1')
df.columns=["sentiment","text"]
df

,sentiment,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
...,...,...
1048566,4,My GrandMa is making Dinenr with my Mum
1048567,4,Mid-morning snack time... A bowl of cheese noo...
1048568,4,@ShaDeLa same here say it like from the Termi...
1048569,4,@DestinyHope92 im great thaanks wbuu?


In [13]:
def preprocess_text(text):
    # Remove URLs, mentions, and special characters
    text = re.sub(r'http\S+|www\S+|@[^\s]+|\W', ' ', text)
    # Convert to lowercase
    text = text.lower()
    return text
df['clean_text']=df['text'].apply(preprocess_text)
df

,sentiment,text,clean_text
0,0,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to ...
2,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why a...
4,0,@Kwesidei not the whole crew,not the whole crew
...,...,...,...
1048566,4,My GrandMa is making Dinenr with my Mum,my grandma is making dinenr with my mum
1048567,4,Mid-morning snack time... A bowl of cheese noo...,mid morning snack time a bowl of cheese noo...
1048568,4,@ShaDeLa same here say it like from the Termi...,same here say it like from the terminiator ...
1048569,4,@DestinyHope92 im great thaanks wbuu?,im great thaanks wbuu


In [6]:
stop_word=set(stopwords.words('english'))
ps=PorterStemmer()
def stemandtoken(text):
    tokens = [ps.stem(word) for word in text if word not in stop_word]
    return tokens
df["t_text"]=df["clean_text"].apply(stemandtoken)
df

,rating,text,clean_text,t_text
0,0,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...,"[ , u, p, e, , h, , h, e, , c, n, , , u, ..."
1,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to ...,"[ , , , v, e, , n, , e, , f, r, , h, e, ..."
2,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[ , w, h, l, e, , b, , f, e, e, l, , c, h, ..."
3,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why a...,"[ , , n, , , , , n, , b, e, h, v, n, g, ..."
4,0,@Kwesidei not the whole crew,not the whole crew,"[ , , n, , h, e, , w, h, l, e, , c, r, e, ..."
...,...,...,...,...
1048566,4,My GrandMa is making Dinenr with my Mum,my grandma is making dinenr with my mum,"[ , g, r, n, , , k, n, g, , n, e, n, r, , ..."
1048567,4,Mid-morning snack time... A bowl of cheese noo...,mid morning snack time a bowl of cheese noo...,"[ , r, n, n, g, , n, c, k, , e, , , , , ..."
1048568,4,@ShaDeLa same here say it like from the Termi...,same here say it like from the terminiator ...,"[ , , e, , h, e, r, e, , , , , l, k, e, ..."
1048569,4,@DestinyHope92 im great thaanks wbuu?,im great thaanks wbuu,"[ , , , g, r, e, , h, n, k, , , w, b, u, ..."


In [6]:
corpus = df['clean_text'].astype(str).tolist()
# Increase min_df value
tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5)
X_tfidf = tfidf_vectorizer.fit_transform(corpus)


X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, df['rating'], test_size=0.2, random_state=42)


In [7]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(X_train,Y_train)
Y_pred=model1.predict(X_test)
print(accuracy_score(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

C:\Users\Abhishek\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8455379920368119
              precision    recall  f1-score   support

           0       0.87      0.94      0.90    159975
           4       0.75      0.53      0.62     49740

    accuracy                           0.85    209715
   macro avg       0.81      0.74      0.76    209715
weighted avg       0.84      0.85      0.84    209715



In [8]:
from sklearn.naive_bayes import MultinomialNB
model2=MultinomialNB()
model2.fit(X_train,Y_train)
Y_pred=model2.predict(X_test)
print(accuracy_score(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

0.804458431681091
              precision    recall  f1-score   support

           0       0.80      0.99      0.89    159975
           4       0.87      0.21      0.33     49740

    accuracy                           0.80    209715
   macro avg       0.84      0.60      0.61    209715
weighted avg       0.82      0.80      0.75    209715



In [ ]:
from sklearn.ensemble import RandomForestClassifier
model3=RandomForestClassifier()
model3.fit(X_train,Y_train)
Y_pred=model3.predict(X_test)
print(accuracy_score(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))